# Sequence to function

## The problem
The problem we are working on is called sequence to function. Ideally this means we have a sequence and we infer some kind of function

### The sequence: 
To me this means a few different things. 
- Gene sequence mutations
- Gene and Protein Orthologs
- Post-translational modification

### The function: 
This also could mean a few things. My mind initially went to physiology, but now realized function could be alot broader. For example simple ligand enzyme binding could be related to funciton. The pathway that the protein is involved in, the regulation
- A change in ligand binding interactions (Initial GPCR activation)
- A change in metabolitic secondary activity (GPCRs downstream)

## The solution
When thinking about a solution to the hackathon problem. Ryan has suggested, could we start with building a full profile with only two data categories (for the sequence). To me this means: 
- Gene name, mutation/ortholog
- Protein name, post-translational modification

Given those could we pull out somekind of knowledge graph that will allow us to relate it to currently known research. This is where we will need the __agent__

In [1]:
import sys, os
sys.path.append("..")

from scripts.fetch_data import fetch_uniprot_data, fetch_genage_data, split_colon_list
from scripts.epmc_utils import fetch_europe_pmc_best, fetch_europe_pmc_best_batch #Haven't named the function yet
from scripts.epmc_utils import list_references_epmc, list_citations_epmc
import pandas as pd
import numpy as np
import pathlib as Path

#TODO: How do we parse the data from here? 

In [2]:
# Maybe here can go the parsing terms
uniprot_data = fetch_uniprot_data(["CCR2"])
display(uniprot_data)

,gene_symbol,uniprot_id,protein_name,sequence,pmids,dois,citation_titles,reviewed
0,CCR2,P41597,C-C chemokine receptor type 2,MLSTSRSRFIRNTNESGEEVTTFFDYDYGAPCHKFDVKQIGAQLLP...,None,None,Molecular cloning and functional expression of...,False


In [3]:
citation_list = split_colon_list(uniprot_data.citation_titles[0])

In [ ]:
for citation in citation_list:
    json_id = fetch_europe_pmc_best(citation_list[0])

In [8]:
list_references_epmc(json_id)

,PMID,PMCID,DOI,title,journal,year,source_url
0,3313277,None,None,An analysis of 5'-noncoding sequences from 699...,None,1987,https://europepmc.org/article/MED/3313277
1,2440339,None,None,Single-step method of RNA isolation by acid gu...,None,1987,https://europepmc.org/article/MED/2440339
2,2923622,None,None,Cloning and sequencing of the cDNA for human m...,None,1989,https://europepmc.org/article/MED/2923622
3,2926329,None,None,Purification and amino acid analysis of two hu...,None,1989,https://europepmc.org/article/MED/2926329
4,2687068,None,None,Macrophage inflammatory proteins 1 and 2: memb...,None,1989,https://europepmc.org/article/MED/2687068
5,2113354,None,None,Cytokine-activated human endothelial cells syn...,None,1990,https://europepmc.org/article/MED/2113354
6,2162890,None,None,Identification of high affinity receptors for ...,None,1990,https://europepmc.org/article/MED/2162890
7,1695010,None,None,Minimally modified low density lipoprotein ind...,None,1990,https://europepmc.org/article/MED/1695010
8,2169649,None,None,An insertion in the human thyrotropin receptor...,None,1990,https://europepmc.org/article/MED/2169649
9,2218504,None,None,Rhodopsin mutants that bind but fail to activa...,None,1990,https://europepmc.org/article/MED/2218504


In [7]:
list_citations_epmc(json_id)

,PMID,PMCID,DOI,title,journal,year,source_url
0,40781135,None,None,MCP-1-CCR2-M2 macrophages axis contributes to ...,None,2025,https://europepmc.org/article/MED/40781135
1,40665002,None,None,Genetically encoded biosensor for monitoring s...,None,2025,https://europepmc.org/article/MED/40665002
2,40457421,None,None,Engineered macrophage membrane-coated dihydroa...,None,2025,https://europepmc.org/article/MED/40457421
3,40406119,None,None,Astrocyte-mediated inflammatory responses in t...,None,2025,https://europepmc.org/article/MED/40406119
4,40342964,None,None,Key gene screening and diagnostic model establ...,None,2025,https://europepmc.org/article/MED/40342964
...,...,...,...,...,...,...,...
497,7531149,None,None,"Actions of the chemotactic cytokines MCP-1, MC...",None,1995,https://europepmc.org/article/MED/7531149
498,7832990,None,None,Isolation of three novel human genes encoding ...,None,1995,https://europepmc.org/article/MED/7832990
499,7548618,None,None,"Chemokines, a family of chemotactic cytokines.",None,1995,https://europepmc.org/article/MED/7548618
500,7961909,None,None,The murine interleukin 8 type B receptor homol...,None,1994,https://europepmc.org/article/MED/7961909
